In [4]:
# ▸ 1. Install & import -------------------------------------------------------
!pip install --quiet krippendorff pandas

import pandas as pd, numpy as np, ast, krippendorff

# ---------------------------------------------------------------------------
files      = ["annotations.csv", "annotations-2.csv", "annotations-3.csv"]
key_col    = "id"         # join key
label_col  = "label"      # label column
# ---------------------------------------------------------------------------

def tidy_labels(cell):
    """
    Cell looks like "['None', 'Minimal']" → 'Minimal|None'.
    Adjust if your label column is already a single string/number.
    """
    lst = ast.literal_eval(cell) if isinstance(cell, str) else [cell]
    return "|".join(sorted(set(map(str, lst))))

dfs = []
for idx, path in enumerate(files, start=1):
    df = pd.read_csv(path, dtype={key_col: str})
    df[label_col] = df[label_col].apply(tidy_labels)
    dfs.append(df[[key_col, label_col]].rename(columns={label_col: f"a{idx}"}))

# Inner join ➜ keep only items every annotator labeled
merged = dfs[0]
for df in dfs[1:]:
    merged = merged.merge(df, on=key_col, how="inner")
if merged.empty:
    raise ValueError("No items have labels from all annotators.")

# Matrix: rows = annotators, cols = items, dtype=str
data = merged[[f"a{i}" for i in range(1, len(dfs)+1)]].to_numpy(dtype=str).T

# Explicit value domain (unique set of labels)
value_domain = pd.unique(data.flatten())

alpha = krippendorff.alpha(reliability_data=data,
                           value_domain=value_domain,   # ← key change
                           level_of_measurement="nominal")  # default

print(f"Krippendorff's alpha: {alpha:.3f}  "
      f"({data.shape[0]} annotators, {data.shape[1]} items)")


Krippendorff's alpha: 0.280  (3 annotators, 50 items)


### PAIRWISE CONFUSION MATRICES

In [6]:
# ▸ 0.  INSTALL / IMPORT (run once per kernel) --------------------------------
!pip install --quiet pandas datasets krippendorff
import pandas as pd, numpy as np, ast, itertools, textwrap
from datasets import load_dataset

# ---------------------------------------------------------------------------
#  (Re-paste the helper functions from the previous cell if they’re not defined)
#   – canonical, pairwise_confusion, display_matrices –
# ---------------------------------------------------------------------------

# ▸ 1.  Local CSVs – SAME as before ------------------------------------------
csv_files = ["annotations.csv", "annotations-2.csv", "annotations-3.csv"]
dfs = []
for idx, path in enumerate(csv_files, start=1):
    df = pd.read_csv(path, dtype={"id": str})
    df["label"] = df["label"].apply(canonical)
    dfs.append(df[["id", "label"]].assign(annotator=f"A{idx}"))

csv_df  = pd.concat(dfs, ignore_index=True)
csv_mats = pairwise_confusion(csv_df)
display_matrices("Local CSV files (≈50 items)", csv_mats)

# ▸ 2.  FineWeb-C Danish – FIXED ---------------------------------------------
fw_ds = load_dataset("data-is-better-together/fineweb-c",
                     name="dan_Latn", split="train")

rows = []
for rec in fw_ds:
    text_id   = str(rec["id"])
    for annot_id, lab in zip(rec["annotator_ids"],
                             rec["educational_value_labels"]):
        rows.append({"id": text_id,
                     "annotator": f"FW{annot_id}",   # prefix for clarity
                     "label": canonical(lab)})

fw_df   = pd.DataFrame(rows)
fw_mats = pairwise_confusion(fw_df, min_overlap=5)   # require ≥5 shared docs
display_matrices("FineWeb-C Danish (~1 k rows after exploding)", fw_mats)




LOCAL CSV FILES (≈50 ITEMS)
 Pair: A1  vs.  A2   (n = 50)


lab_b,Basic,Excellent,Good,Minimal,Minimal|None,None,Problematic
lab_a,,,,,,,
Basic,0,0,0,1,0,1,0
Excellent,1,0,1,0,0,0,0
Good,1,0,0,0,0,1,0
Minimal,1,1,0,8,0,7,0
None,2,0,0,4,1,14,2
Problematic,0,0,0,0,0,0,4


 Pair: A1  vs.  A3   (n = 50)


lab_b,Basic,Good,Minimal,None,Problematic
lab_a,,,,,
Basic,0,0,0,2,0
Excellent,1,1,0,0,0
Good,0,0,1,1,0
Minimal,0,0,4,13,0
None,1,0,1,21,0
Problematic,0,0,0,1,3


 Pair: A2  vs.  A3   (n = 50)


...

lab_b,Basic,Excellent,Good,Minimal,None
lab_a,,,,,
Minimal,5,2,2,3,0
None,2,0,1,7,5
❗ Problematic Content ❗,0,0,0,0,6


 Pair: FW85ac8d54-89c5-4473-95c4-797366f03cd0  vs.
FW29b60230-70be-4917-aadd-994555f53aed   (n = 25)


lab_b,Basic,Minimal,None,❗ Problematic Content ❗
lab_a,,,,
Basic,1,1,0,0
Minimal,0,2,4,0
None,1,2,12,0
❗ Problematic Content ❗,0,0,0,2


 Pair: FW85ac8d54-89c5-4473-95c4-797366f03cd0  vs.  FW288ddcda-
ec4c-4fe3-8866-8ae5058dc15b   (n = 5)


lab_b,None
lab_a,
Minimal,2
None,2
❗ Problematic Content ❗,1
